<a href="https://colab.research.google.com/github/deybvagm/natural-language-processing/blob/master/fastext_yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de lenguaje natural con el conjunto de datos Yelp y la librería fasttest

[Yelp](https://www.yelp.com/)  es una plataforma que permite a usuarios hacer reviews de servicios ofrecidos por empresas. Por ejemplo se puede encontrar reviews de restaurantes, parqueaderos, empresas que ofrecen servicios para el hogar como plomería, electricistas, entre otros. Yelp ha proporcionado una base de datos de 4.7 millones re reviews para que pueda srvir como referencia cuando se trabaja con modelos de procesamiento de lenguaje natural. 

[Fasttext](https://fasttext.cc/) es una librería de procesamiento de lenguaje natural creada por el laboratorio de inteligencia artificial de facebook. En este caso específico, será utilizada para clasificar los reviews de usuarios en una de 5 posibles categorias (numero de estrellas con el que el usuario valora un producto o servicio en Yelp)

### Instalación de librería para procesamiento de texto

In [0]:
! pip install fasttext

     |████████████████████████████████| 61kB 1.2MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2382781 sha256=faf1a0a3398deb4c386b65c882507eefecf08a59b53ccd636754481ed76dc769
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext


### Integración con Google Drive
Es de utilidad ya que el modelo esta almacenado en Drive y por lo tanto para hacer uso de él podemos hacer esta integración con el fin de acceder a los datos que allí se tengan

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Ahora nos vamos a la carpeta de Drive donde está almacenado el modelo

In [0]:
%cd drive/My\ Drive/nlp

/content/drive/My Drive/nlp


In [0]:
! ls

 aclImdb_pred   proc_data		     'xlnet commands.txt'
 exp	        xlnet			      yelp
 predictions    xlnet_cased_L-12_H-768_A-12   yelp_model


### Import de librerías a usar y definición de funciones auxiliares para pre-procesamiento de texto

La función auxiliar es encargada de convertir el texto a minúsculas y además de dejar el texto en un formato que pueda ser leído por la librería `fasttext`.  Por ejemplo un texto como **`El restaurante esmuy bueno!`** es  convertido en **`__label__5 el restaurante es muy bueno!`**

In [0]:
import fasttext
import re

def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

### Entrenamiento del modelo de Machine Learning

Para entrenar el modelo de machine learning se requiere la información de las reviews de clientes junto con su calificación (número de estrellas). Además de esto es necesario dividir los datos en dos conjuntos, uno de entrenamiento con el cual se realiza el aprendizaje, y el otro de pruebas usado para hacer la evaluación del modelo obtenido. Una vez realizado el entrenamiento, se guarda el modelo en una ruta indicada (en este caso una ruta en Google Drive)

In [0]:
import json
from pathlib import Path
import random

reviews_data = Path("yelp") / "review.json"
training_data = Path("yelp") / "fasttext_dataset_training.txt"
test_data = Path("yelp") / "fasttext_dataset_test.txt"

# What percent of data to save separately as test data
percent_test_data = 0.10

with reviews_data.open() as input, \
     training_data.open("w") as train_output, \
     test_data.open("w") as test_output:

    for line in input:
        review_data = json.loads(line)

        rating = review_data['stars']
        text = review_data['text'].replace("\n", " ")
        text = strip_formatting(text)

        fasttext_line = "__label__{} {}".format(rating, text)

        if random.random() <= percent_test_data:
            test_output.write(fasttext_line + "\n")
        else:
            train_output.write(fasttext_line + "\n")

In [0]:
model = fasttext.train_supervised('yelp/fasttext_dataset_training.txt', word_ngrams=2)


In [0]:
model.save_model('yelp_model/yelp_model.bin')

In [0]:
!ls

aclImdb			       proc_data
aclImdb_pred		       xlnet
aclImdb_v1.tar.gz	       xlnet_cased_L-12_H-768_A-12
cased_L-12_H-768_A-12.zip      xlnet_cased_L-24_H-1024_A-16
cased_L-24_H-1024_A-16.zip     yelp
exp			       yelp_dataset.tar
fasttext_dataset_training.txt  yelp_model.bin
predictions


### Predicciones sobre el modelo

Con el modelo creado y almacenado en disco, ahora es posible realizar predicciones sobre nuevos reviews para identificar automáticamente el número de estrellas que el modelo asignaría a los reviews. Es importante que para los nuevos reviews se haga un pre-procesamiento de los datos para dejarlos en el formato que exige la librería

In [0]:
# Reviews to check
reviews = [
    "This restaurant literally changed my life. This is the best food I've ever eaten!",
    "I hate this place so much. They were mean to me.",
    "I don't know. It was ok, I guess. Not really sure what to say."
]

# Pre-process the text of each review so it matches the training format
preprocessed_reviews = list(map(strip_formatting, reviews))

# Load the model
classifier = fasttext.load_model('yelp_model/yelp_model.bin')

# Get fastText to classify each review with the model
labels, probabilities = classifier.predict(preprocessed_reviews, 1)

# Print the results
for review, label, probability in zip(reviews, labels, probabilities):
    stars = int(label[0][-3])

    print("{} stars ({}% confidence)".format(stars, int(probability[0] * 100)))
    print(review)
    print()

5 stars (100% confidence)
This restaurant literally changed my life. This is the best food I've ever eaten!

1 stars (98% confidence)
I hate this place so much. They were mean to me.

3 stars (69% confidence)
I don't know. It was ok, I guess. Not really sure what to say.



### Descarga del modelo

La siguiente instrucción realiza la descarga del modelo

In [0]:
from google.colab import files
files.download('yelp/yelp_model.bin') 

### ¿Qué tan bueno es el modelo para realizar predicciones?

El siguiente paso es determinar la capacidad del modelo para hacer predicciones sobre reviews que no ha visto. Para esto se usa el conjunto de datos de prueba

In [0]:
eval = classifier.test('yelp/fasttext_dataset_test.txt')
print(eval)

(668564, 0.7293123769751287, 0.7293123769751287)


El resultado nos dice que se evaluaron 668.564 reviews, de las cuales el modelo dice que el 72% de ellas fueron exactamente bien clasificadas de acuerdo al número de estrellas. 

Tambien podemos decirle al modelo que para cualquier review, me de dos posibles opciones de respuesta (numero de estrellas) que él crea se ajustan más. Por ejemplo "El review .... podría ser de 4 o 5 estrellas"

In [0]:
eval = classifier.test('yelp/fasttext_dataset_test.txt', 2)
print(eval)

(668564, 0.4702788364315159, 0.9405576728630318)


El resultado nos dice que se evaluaron 668.564 reviews, de las cuales el modelo dice que el 94% de ellas fueron bien clasificadas de acuerdo a las dos predicciones que el cree más aproximadas.